In [1]:
import os,sys, json
import magic, hashlib, os, traceback
import ntpath
from capstone import *
from capstone.x86 import *

import collections
import traceback
from elftools.elf.elffile import ELFFile

In [2]:
bin_path = '/home/raisul/DATA/x86_O2_d4/' #x86_O2_d4_mingw32_PE

bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) ][0:100]

ground_truth_path = '/home/raisul/ANALYSED_DATA/objdump_x86_O2_d4/'

In [3]:
len(bin_files)

100

In [4]:


def get_ground_truth(exe_path, text_section_offset , text_section_len ):

    text_sextion_end = text_section_offset + text_section_len
    
    elf_file_name = os.path.basename(exe_path)
    objdump_file_path = os.path.join(ground_truth_path, elf_file_name.split('.')[0]) + '.json'
    
    with open(objdump_file_path, "r") as file:
        objdump_data = json.load(file)

    ground_truth_offsets = list(objdump_data["instructions"].keys())
    ground_truth_offsets.remove('meta')
    ground_truth_offsets = [int(i) for i in ground_truth_offsets]
    ground_truth_offsets = [x for x in ground_truth_offsets if text_section_offset <= x <= text_sextion_end]
    ground_truth_offsets.sort()

    
    return ground_truth_offsets

In [5]:



def find_data_in_textsection(ground_truth_offsets , text_section_offset , text_section_len, offset_inst_dict):

    for i in range(1, len(ground_truth_offsets)):
        gap = ground_truth_offsets[i] - ground_truth_offsets[i-1]

        inst_len = offset_inst_dict[ground_truth_offsets[i-1]].size 
        
        if gap!=inst_len:
            print('offset_ranges[i]: ',ground_truth_offsets[i] , 'offset_ranges[i-1]: ',ground_truth_offsets[i-1], ' inst_len: ',inst_len  ) 
        else:
            # print('hi')
            pass


def linear_sweep(offset_inst , target_offset):
    inst_sequence = []
    max_seq_length = 15

    
    current_offset = target_offset
    for q in range(max_seq_length):

        if current_offset in offset_inst: #if end of text section
            current_instruction = offset_inst[current_offset]
            if current_instruction is None:

                inst_sequence.insert(0, None) 
                inst_sequence.append(None)
                return  inst_sequence
                
            current_offset = current_offset + current_instruction.size
            inst_sequence.append( str(current_instruction.address) +": "+ current_instruction.mnemonic +' '+ current_instruction.op_str )
            # inst_sequence.append( current_instruction)
            if current_instruction.mnemonic in ["ret", "jmp"]: #break linear sweep
                return inst_sequence

    return inst_sequence


for bin_file_path in bin_files:

    # try:
    #     ground_truth_offsets = get_ground_truth(bin_file_path, text_section_offset , len(textSection))
    # except Exception as e:
    #     print("An error occurred:", e)
    # print(ground_truth_offsets)
    
    
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    # print("\n"*5)
    
    with open(bin_file_path, 'rb') as f:

        try:
            elffile = ELFFile(f)
            print('1')
            textSection = elffile.get_section_by_name('.text').data()
            print('2')
            text_section_offset = elffile.get_section_by_name('.text')['sh_offset']
            print('3')
            ground_truth_offsets = get_ground_truth(bin_file_path, text_section_offset , len(textSection))
            
        except Exception as e:
            print("An error occurred:", e ,bin_file_path)
        for byte_index in range(len(textSection)):
            
        
            try:    

                instruction = next(md.disasm(textSection[byte_index: byte_index+15 ], text_section_offset + byte_index ), None)
                
                # if instruction:
                #     print("%d:\t%s\t%s _\t%x" %(int(instruction.address), instruction.mnemonic, instruction.op_str, instruction.size))
                #     print(instruction.operands[0].type , instruction.operands[0].imm)
                #     # print( dir(instruction.operands[0] ))
          
                # else:
                #     print("%d:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )
                    
                offset_inst[text_section_offset+byte_index] = instruction

            except Exception as e:
                print(traceback.print_exc() )
                print(e)

        
        
        offset_inst = collections.OrderedDict(sorted(offset_inst.items()))

        find_data_in_textsection(ground_truth_offsets , text_section_offset , len(textSection) , offset_inst)
        
        # print(offset_inst)
        for byte_index in range(len(textSection)):
            inst_seq = linear_sweep(offset_inst ,text_section_offset + byte_index )
            # print('\n','# # '*10)
            # print(inst_seq)
    # print(ground_truth_offsets)
    # print(offset_inst.keys())
    print('\n','# # '*30)


1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
1
2
3

 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

In [6]:
print('test')

test
